In [ ]:
pip install pickle5

In [ ]:
import pandas as pd
import numpy as np 
import pickle5 as pickle
import collections

In [ ]:
open_file = open('/content/drive/MyDrive/filename.pkl', "rb")
dic = pickle.load(open_file)

In [ ]:
open_file = open('/content/drive/MyDrive/First_file_new.pickle', "rb")
data = pickle.load(open_file)

In [ ]:
data.shape


(356124, 16)

In [ ]:
dic

{'acer': ['acer-liquid-z6-plus', 'acer-liquid-z630'],
 'alcatel': ['alcatel-onetouch-20-04',
  'alcatel-idol-3',
  'alcatel-onetouch-pop-3-5015d',
  'alcatel-idol-3-5-5-6045y',
  'alcatel-onetouch-pixi-4',
  'alcatel-one-touch-idol-4s',
  'alcatel-one-touch-2004c'],
 'apple': ['apple-iphone-7',
  'apple-iphone-6s-plus',
  'apple-iphone-se',
  'apple-iphone-6-plus',
  'apple-iphone-7-plus',
  'apple-iphone-6s',
  'apple-iphone-6'],
 'asus': ['asus-zenfone-2-laser-ze500kl',
  'asus-zenfone-3',
  'asus-zenfone-3-deluxe',
  'asus-zenfone-3-zoom',
  'asus-zenfone-2-ze551ml',
  'asus-zenfone-max-zc550kl',
  'asus-zenfone-3-ultra',
  'asus-zenfone-go-zc500tg',
  'asus-zc551kl',
  'asus-zenfone-3-max',
  'asus-ze500kl',
  'asus-ze550kl',
  'asus-zenfone-2',
  'asus-zenfone-selfie-zd551kl',
  'asus-zenfone-2-laser-ze601kl',
  'asus-zenfone-zoom'],
 'blackberry': ['blackberry-leap',
  'blackberry-priv',
  'blackberry-passport',
  'blackberry-dtek60',
  'blackberry-venice',
  'blackberry-dtek50']

In [ ]:
import os.path

save_path = '/content/drive/MyDrive/New_pickles'

name_of_file = 'test'

completeName = os.path.join(save_path, name_of_file+".txt")         

file1 = open(completeName, "w")

toFile = 'this is working'
file1.write(toFile)

file1.close()

In [ ]:
import spacy
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob
import operator
nlp = spacy.load("en_core_web_sm") 

In [ ]:
def generate_text(new_df):
  text= ""
  for i in new_df['extract']:
    text+=str(i)
  return text

In [ ]:
def ABSA(text):
  tok_text = nltk.tokenize.sent_tokenize(text)
  aspect = {}
  polarity_aspect = {}
  num_aspect = {}
  def number(target):
    if target in num_aspect:
        num_aspect[target] += 1
    else:
        num_aspect[target] = 1
  def polarity(target,descriptive_term):
    k = TextBlob(descriptive_term).sentiment.polarity
    if target in polarity_aspect:
      if k<=0:
        polarity_aspect[target][0]+=1
      else:
        polarity_aspect[target][1]+=1
                    
    else:
      polarity_aspect[target] = [0,0]
      if k<=0:
          polarity_aspect[target][0]+=1
      else:
          polarity_aspect[target][1]+=1
  def add_aspect(target,descriptive_term):
    if target in aspect:
        aspect[target].add(descriptive_term)
    else:
        aspect[target]= {descriptive_term}
  for sentence in tok_text:
    doc = nlp(str(sentence))
    descriptive_term = ''
    target = ''
    for token in doc:
      if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
        target = token.text
      if token.pos_ == 'ADJ':
        prepend = ''
        for child in token.children:
          if child.pos_ != 'ADV':
            continue
          prepend += child.text + ' '
        descriptive_term = prepend + token.text
      if target != '' and descriptive_term != '':
        target = target.lower()
        descriptive_term = descriptive_term.lower()
        number(target)
        polarity(target,descriptive_term)
        add_aspect(target,descriptive_term)
      else:
        pass
        
  return aspect, polarity_aspect, num_aspect

In [ ]:
def list_operation(aspect, polarity_aspect, num_aspect):
  remove_set = {'d','husband','wife','thing','people'}
  sorted_d = dict( sorted(num_aspect.items(), key=operator.itemgetter(1),reverse=True))
  for i in remove_set:
    if i in sorted_d:
        sorted_d.pop(i)
    else:
        pass
  target_list = []
  if len(sorted_d)<15:
    k = len(sorted_d)
    for i in sorted_d:
      target_list.append(i)
      k-=1
      if k == 0:
        break
  else:
    k = 15
    for i in sorted_d:
      target_list.append(i)
      k-=1
      if k == 0:
        break
  sentiment_list = []
  sentiment = []
  for i in target_list:
    sentiment_list.append(list(aspect[i])[:15])
    sentiment.append(polarity_aspect[i])    
  
  return target_list,sentiment_list,sentiment


In [ ]:
def save_file(name,target_list,sentiment_list,sentiment):
  T = [target_list,sentiment_list,sentiment]
  save_path = '/content/drive/MyDrive/New_pickles'
  name_of_file = name
  completeName = os.path.join(save_path, name_of_file+".pkl")
  file1 = open(completeName, "wb")
  pickle.dump(T, file1)
  file1.close()




In [ ]:
#from nltk.internals import java
import os
k =1
for i in dic:
  print(k)
  for j in dic[i]:
    print(j)
    new_df = data[data['Phone_names'] == j]
    text = generate_text(new_df)
    aspect,polarity_aspect,num_aspect = ABSA(text)
    target_list,sentiment_list,sentiments = list_operation(aspect,polarity_aspect,num_aspect)
    save_file(j,target_list,sentiment_list,sentiments)
  k+=1




1
samsung-galaxy-j7
samsung-galaxy-a3-2016
samsung-galaxy-note-4
samsung-galaxy-e5
samsung-galaxy-e7
samsung-galaxy-s7-789999
samsung-galaxy-s6
samsung-galaxy-a9-pro-dual-sm-a9100
samsung-galaxy-a7-2016
samsung-galaxy-grand-max
samsung-galaxy-s5
samsung-galaxy-j1-2016-4-5-sm-j120
samsung-galaxy-j5-4g-lte-5-2-inches
samsung-galaxy-a9
samsung-galaxy-s6-active-sm-g890
samsung-galaxy-j7-prime
samsung-galaxy-s6-edgeplus
samsung-galaxy-alpha
samsung-galaxy-s7-active
samsung-galaxy-j5-prime-sm-g570f
samsung-galaxy-s8
samsung-galaxy-s5-active-g870
samsung-galaxy-a7
samsung-galaxy-c7
samsung-galaxy-s7-edge
samsung-galaxy-note-edge
samsung-galaxy-j5
samsung-galaxy-j7-2016
samsung-galaxy-a5
samsung-galaxy-s5-sport-sm-g860p
samsung-galaxy-s8-plus
samsung-galaxy-a3-819970
samsung-galaxy-s5-neo-sm-g903f
samsung-galaxy-s6-edge-sm-g925f
samsung-galaxy-a5-2016
samsung-galaxy-grand-prime
samsung-galaxy-a5-2017
samsung-galaxy-on7
samsung-galaxy-a3
samsung-z
samsung-galaxy-grand-prime-ve-sm-g531f
samsung-

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
new_df = data[data['Phone_names'] == 'samsung-galaxy-j7']
text = generate_text(new_df)
aspect,polarity_aspect,num_aspect = ABSA(text)
target_list,sentiment_list,sentiment = list_operation(aspect,polarity_aspect,num_aspect)
#save_file('samsung-galaxy-j7',target_list,sentiment_list,sentiment)

In [ ]:
target_list

['phone',
 'camera',
 'product',
 'battery',
 'quality',
 'screen',
 'device',
 'volume',
 'mobile',
 'life',
 'problem',
 'display',
 'performance',
 'features',
 'sound']

In [ ]:
sentiment_list

[['supergreat',
  'harder',
  'amazed',
  'intermediate',
  'sleek',
  'so incredibly low',
  'upgradable',
  'removable',
  'very similar',
  'disappointed',
  'massive',
  'really good',
  'clean',
  'speedy',
  'fantastic'],
 ['cheap',
  'natural',
  '13mp',
  'really good',
  'clean',
  'very grainy).again',
  'fantastic',
  'very fast',
  'so good',
  'only',
  'light',
  'low',
  'perfectly good',
  'fairly large',
  'back'],
 ['pricegreat',
  'very goodvery',
  'intermediate',
  'sure',
  'dummy',
  'disappointed',
  'very fast',
  'available',
  'light',
  'low',
  'comfortable',
  'efficient',
  'personal',
  'functional',
  'able'],
 ['greatafter',
  'impressed',
  'sure',
  'removable',
  'really good',
  'fantastic',
  'entire',
  'only',
  'low',
  'non',
  'expensive',
  'several',
  'online',
  'able',
  'regular'],
 ['disappointed',
  'very fast',
  'quite good',
  'only',
  'light',
  'too bad',
  'simply fantastic',
  'low',
  'prevalent',
  'unbelievable',
  'able',


In [ ]:
sentiment

[[1832, 2926],
 [460, 888],
 [331, 933],
 [482, 514],
 [273, 602],
 [306, 348],
 [201, 301],
 [212, 216],
 [114, 245],
 [79, 253],
 [196, 111],
 [66, 241],
 [129, 171],
 [85, 178],
 [143, 111]]

In [ ]:
save_file('samsung-galaxy-j7',target_list,sentiment_list,sentiment)